### First we need to import the right packages

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 6.2MB/s eta 0:00:01


### The Beautiful Soup Package parses through the Wikipedia page and creates data for us.

In [2]:
# Using Beautiful Soup to parse the website's html
data = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans').text
soup = BeautifulSoup(data, 'lxml')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Neighborhoods in New Orleans - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f7cfbc4e-7498-4991-80b1-e7e49ee917f6","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Neighborhoods_in_New_Orleans","wgTitle":"Neighborhoods in New Orleans","wgCurRevisionId":951143819,"wgRevisionId":951143819,"wgArticleId":3950865,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from April 2020","Ar

### Next we will use pandas to create a table from the "soup".

In [3]:
import pandas as pd
Table = soup.find('table', {'class':'wikitable sortable'})
df = pd.read_html(str(Table))
df = df[0]
df.iat[4, 1]= -89.9953 #Correction on an incorect Longitude for Old Aurora
df.head(30)

,Neighborhood,Longitude,Latitude
0,U.S. NAVAL BASE,-90.026093,29.946085
1,ALGIERS POINT,-90.051606,29.952462
2,WHITNEY,-90.042357,29.947200
3,AUDUBON,-90.121450,29.932994
4,OLD AURORA,-89.995300,29.924440
5,B. W. COOPER,-90.091753,29.951774
6,BAYOU ST. JOHN,-90.086517,29.976071
7,BEHRMAN,-90.026436,29.934817
8,BLACK PEARL,-90.134883,29.935895
9,BROADMOOR,-90.103812,29.946568


### Importing all of the libraries that we need

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

### Next the table of the neighborhoods will be mapped.

In [5]:
import folium

# create map of New Orleans using latitude and longitude values
map_neworleans = folium.Map(location=[29.9511, -90.0715], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neworleans)  
    
map_neworleans

### Now we will use foursquare to gather more information about the neighborhoods.

In [6]:
CLIENT_ID = 'WTMF50XDVEP05D5A0EH2TMJDYFU2EFPYFTQUTQ0U5RJ31UCN'
CLIENT_SECRET = 'H4WYMCOVSE0EXMFSXTQX1I1DNQ3RLHX0APL3PAMHUKDMDHML'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WTMF50XDVEP05D5A0EH2TMJDYFU2EFPYFTQUTQ0U5RJ31UCN
CLIENT_SECRET:H4WYMCOVSE0EXMFSXTQX1I1DNQ3RLHX0APL3PAMHUKDMDHML


In [7]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of U.S. NAVAL BASE are 29.94608469, -90.02609253.


In [8]:
# type your answer here
LIMIT = 100
radius= 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4303d41883f35ebf705d56'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Algiers',
  'headerFullLocation': 'Algiers, New Orleans',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 29.950584694500005,
    'lng': -90.02090888810879},
   'sw': {'lat': 29.941584685499993, 'lng': -90.03127617189121}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e330feaaeb7226862583dc7',
       'name': 'The Mighty Missisippi',
       'location': {'lat': 29.949695231438263,
        'lng': -90.0237103861832,
        'labeledLatLngs': [{'label': 'display',
          'lat': 2

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
neworleans_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST. ANTHONY
LAKE TERRACE & OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


### In order to make this data usable, the one hot technique will be used to create dummy variables.

In [14]:
# one hot encoding
neworleans_onehot = pd.get_dummies(neworleans_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neworleans_onehot['Neighborhood'] = neworleans_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(neworleans_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
neworleans_onehot=neworleans_onehot[['Neighborhood']+cols]

neworleans_onehot.head(5)

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Light Rail Station,Liquor Store,Locksmith,Lounge,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Travel & Transport,Vegetarian / Vegan Restaurant,Vehicle Inspection Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,U.S. NAVAL BASE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [15]:
neworleans_grouped = neworleans_onehot.groupby('Neighborhood').mean().reset_index()
neworleans_grouped #Combining all of the data for each neighborhood

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Light Rail Station,Liquor Store,Locksmith,Lounge,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Travel & Transport,Vegetarian / Vegan Restaurant,Vehicle Inspection Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,ALGIERS POINT,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.130435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.173913,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.086957,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.043478,0.043478,0

### Finding out the top 5 venues for each of these neighborhoods.

In [16]:
num_top_venues = 5

for hood in neworleans_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neworleans_grouped[neworleans_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALGIERS POINT----
           venue  freq
0  Boat or Ferry  0.17
1            Bar  0.13
2    Coffee Shop  0.09
3  Grocery Store  0.04
4    Pizza Place  0.04


----AUDUBON----
                   venue  freq
0  Outdoors & Recreation  0.14
1           Concert Hall  0.14
2         Sandwich Place  0.14
3  College Arts Building  0.14
4            Coffee Shop  0.14


----B. W. COOPER----
                  venue  freq
0            Boxing Gym   0.2
1     Recreation Center   0.2
2            Food Truck   0.2
3  Gym / Fitness Center   0.2
4          Home Service   0.2


----BAYOU ST. JOHN----
                  venue  freq
0           Gas Station  0.14
1     Convenience Store  0.14
2  Other Great Outdoors  0.14
3        Sandwich Place  0.14
4                  Park  0.14


----BLACK PEARL----
               venue  freq
0      Grocery Store  0.25
1         Food Truck  0.25
2               Pool  0.25
3     Farmers Market  0.25
4  Accessories Store  0.00


----BROADMOOR----
               venue  fr

### Next a table of the top 10 venues in each neighborhood will be created.

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neworleans_grouped['Neighborhood']

for ind in np.arange(neworleans_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neworleans_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Boat or Ferry,Bar,Coffee Shop,Music Store,Pizza Place,Gift Shop,Park,Grocery Store,Music Venue,Scenic Lookout
1,AUDUBON,Concert Hall,College Arts Building,Coffee Shop,Plaza,Sandwich Place,Smoothie Shop,Outdoors & Recreation,Yoga Studio,Event Space,Food Court
2,B. W. COOPER,Recreation Center,Gym / Fitness Center,Food Truck,Home Service,Boxing Gym,French Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
3,BAYOU ST. JOHN,Lounge,Convenience Store,Gas Station,Other Great Outdoors,Sandwich Place,Playground,Park,Event Space,Food Court,Food & Drink Shop
4,BLACK PEARL,Food Truck,Pool,Grocery Store,Farmers Market,Yoga Studio,Event Space,Frame Store,Food Court,Food & Drink Shop,Food


### Using K Means clustering to create three similar groups. From here we can decide which cluster is most appealing to locate a restaurant.

In [19]:
# set number of clusters
kclusters = 3

neworleans_grouped_clustering = neworleans_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neworleans_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int32)

In [20]:
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

In [21]:
neworleans_merged = df
neworleans_merged=neworleans_merged.dropna()
# merge neworleans_grouped with neworleans_data to add latitude/longitude for each neighborhood
neworleans_merged = neworleans_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
neworleans_merged=neworleans_merged.dropna()
neworleans_merged['Cluster_Labels'] = neworleans_merged.Cluster_Labels.astype(int)
neworleans_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,1,Boat or Ferry,Yoga Studio,Garden,Fried Chicken Joint,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food
1,ALGIERS POINT,-90.051606,29.952462,1,Boat or Ferry,Bar,Coffee Shop,Music Store,Pizza Place,Gift Shop,Park,Grocery Store,Music Venue,Scenic Lookout
2,WHITNEY,-90.042357,29.947200,1,Convenience Store,Nightclub,Credit Union,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
3,AUDUBON,-90.121450,29.932994,1,Concert Hall,College Arts Building,Coffee Shop,Plaza,Sandwich Place,Smoothie Shop,Outdoors & Recreation,Yoga Studio,Event Space,Food Court
4,OLD AURORA,-89.995300,29.924440,2,Nightlife Spot,Yoga Studio,Indie Theater,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


### Mapping the clusters

In [22]:
# create map
map_clusters = folium.Map(location=[29.9511, -90.0715], zoom_start=11) #Latitude, Longitude

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neworleans_merged['Latitude'], neworleans_merged['Longitude'], neworleans_merged['Neighborhood'], neworleans_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=2).add_to(map_clusters)
       
map_clusters

In [23]:
neworleans_merged.loc[neworleans_merged['Cluster_Labels'] == 0, neworleans_merged.columns[[1] + list(range(4, neworleans_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,-90.097847,Park,Harbor / Marina,Yoga Studio,Event Space,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
55,-90.039310,Park,Yoga Studio,Indie Theater,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


In [24]:
neworleans_merged.loc[neworleans_merged['Cluster_Labels'] == 1, neworleans_merged.columns[[1] + list(range(4, neworleans_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-90.026093,Boat or Ferry,Yoga Studio,Garden,Fried Chicken Joint,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food
1,-90.051606,Boat or Ferry,Bar,Coffee Shop,Music Store,Pizza Place,Gift Shop,Park,Grocery Store,Music Venue,Scenic Lookout
2,-90.042357,Convenience Store,Nightclub,Credit Union,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
3,-90.121450,Concert Hall,College Arts Building,Coffee Shop,Plaza,Sandwich Place,Smoothie Shop,Outdoors & Recreation,Yoga Studio,Event Space,Food Court
5,-90.091753,Recreation Center,Gym / Fitness Center,Food Truck,Home Service,Boxing Gym,French Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
6,-90.086517,Lounge,Convenience Store,Gas Station,Other Great Outdoors,Sandwich Place,Playground,Park,Event Space,Food Court,Food & Drink Shop
8,-90.134883,Food Truck,Pool,Grocery Store,Farmers Market,Yoga Studio,Event Space,Frame Store,Food Court,Food & Drink Shop,Food
9,-90.103812,Music Venue,Recreation Center,Food Truck,Coffee Shop,Taco Place,Farmers Market,French Restaurant,Frame Store,Food Court,Food & Drink Shop
10,-90.113468,Bar,Nightclub,Food & Drink Shop,Campground,Cocktail Bar,Lounge,Farmers Market,French Restaurant,Frame Store,Food Truck
11,-90.105829,Locksmith,Seafood Restaurant,Gas Station,Asian Restaurant,Financial or Legal Service,Tennis Court,Bakery,Frame Store,Food Truck,Food Court


In [25]:
neworleans_merged.loc[neworleans_merged['Cluster_Labels'] == 2, neworleans_merged.columns[[1] + list(range(5, neworleans_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,-89.9953,Yoga Studio,Indie Theater,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


### We have all of the data needed. Now we need to use the data for analysis.